<center>
<h1>Python Application Building</h1>
<img src="http://i.imgur.com/91PUPZA.png" width=20%>
</center>

<center>
AY 250, Spring 2013, Fall 2016; Josh Bloom
</center>

In [1]:
!pip install simplejson --target=/tmp/

    100% |████████████████████████████████| 71kB 365kB/s 


In [2]:
!ls /tmp/simple*

/tmp/simplejson:
__init__.py                     encoder.py
__pycache__                     ordered_dict.py
_speedups.cpython-35m-darwin.so scanner.py
compat.py                       tests
decoder.py                      tool.py

/tmp/simplejson-3.10.0.dist-info:
DESCRIPTION.rst METADATA        WHEEL           top_level.txt
INSTALLER       RECORD          metadata.json


`...dist-info` directory is part of the "standard infrastructure to manage project distributions installed on a system, so all tools that are installing or removing projects are interoperable." [PEP-376](https://www.python.org/dev/peps/pep-0376/#id19)

[Note the .whl extension...see ["Wheel vs EGG"](https://packaging.python.org/wheel_egg/) and [Wheel info](https://wheel.readthedocs.io/en/latest/)]

Now, you can have Python know about your special installation directory by modifying your `PYTHONPATH` environment variable in your `.bashrc`, `.cshrc`, or `.tcshrc` file:
```bash
#BASH Style: 
export PYTHONPATH=/tmp/simplejson:$PYTHONPATH
#CSH Style:
setenv PYTHONPATH /path/to/my_choice:$PYTHONPATH
```

# Getting and Installing Packages with `setup.py` #

Sometimes `conda` and `pip` cannot find a codebase you're trying to install. In this case you'll need to do it yourself using a tarball and a `setup.py` file. This is the most straightforward way to get packages: download them from the developer’s website and hope that they’ve followed the standard conventions.

There is a standard Python package distribution scheme using `distutils2` and `setup.py` files...more on that later.

Basic workflow of installing a package with `setup.py`:
```bash
$ cd [folder with package and setup.py file]
$ sudo python setup.py install
   # [ progress report ... ]
$ Finished processing dependencies for [package]
   # [if you want more info, there are several options to modify]
$ python setup.py --help install
```

To do a custom installation directory (if you dont have sudo, e.g.):
```bash
# {-- on unix --}
$ python setup.py install --home <my_choice>

# {-- on windows --}
$ python setup.py install --prefix “my_choice”
```

# Managing Packages - `virtualenv`/`conda` environments #

* Open Source software is constantly changing - how do you protect working code against future updates?
* Or, what if there is a beta release of a package you want to try, but you don’t want to fully commit yet?
* `virtualenv` and `conda -n` creates a local, self-contained, and totally separate python installation.
* Use it to create a local Python ecosystem, separate from your computer’s main system, so that you can do what you want in one without affecting the other.

# `virtualenv` #

installing:

In [3]:
!pip install --upgrade virtualenv

    100% |████████████████████████████████| 1.8MB 178kB/s 


Creating a new environment:

[![asciicast](https://asciinema.org/a/cxhbq03rtd39ez8sy5scwqk1w.png)](https://asciinema.org/a/cxhbq03rtd39ez8sy5scwqk1w)

(https://asciinema.org/)

During a shell session, you can source this environment so that it runs as the default:

```bash
$ source LocalPython/bin/activate
(LocalPython)$
#[ pip and python commands now point to new environment ]
(LocalPython)$ which python
LocalPython/bin/python
```

We can get out of the enviroment:

```bash
 deactivate LocalPython
```
Just delete to remove environment:

```bash
rm -r LocalPython
```


# conda -n #
http://www.continuum.io/blog/conda

In [1]:
!conda info


             platform : osx-64
        conda version : 4.1.11
    conda-env version : 2.5.2
  conda-build version : 1.9.1
       python version : 2.7.11.final.0
     requests version : 2.10.0
     root environment : /Users/jbloom/anaconda  (writable)
  default environment : /Users/jbloom/anaconda/envs/seminar
     envs directories : /Users/jbloom/anaconda/envs
        package cache : /Users/jbloom/anaconda/pkgs
         channel URLs : https://repo.continuum.io/pkgs/free/osx-64/
                        https://repo.continuum.io/pkgs/free/noarch/
                        https://repo.continuum.io/pkgs/pro/osx-64/
                        https://repo.continuum.io/pkgs/pro/noarch/
          config file : None
         offline mode : False
    is foreign system : False



In [2]:
!conda info -e

#
_test                    /Users/jbloom/anaconda/envs/_test
bootcamp                 /Users/jbloom/anaconda/envs/bootcamp
dd                       /Users/jbloom/anaconda/envs/dd
i13                      /Users/jbloom/anaconda/envs/i13
ipykernel_py2            /Users/jbloom/anaconda/envs/ipykernel_py2
numpy15                  /Users/jbloom/anaconda/envs/numpy15
py3k                     /Users/jbloom/anaconda/envs/py3k
seminar               *  /Users/jbloom/anaconda/envs/seminar
root                     /Users/jbloom/anaconda



In [4]:
!conda search simplejson

Fetching package metadata .......
simplejson                   3.8.1                    py27_0  defaults        
                             3.8.1                    py34_0  defaults        
                             3.8.1                    py35_0  defaults        
                             3.8.2                    py27_0  defaults        
                             3.8.2                    py34_0  defaults        
                             3.8.2                    py35_0  defaults        
                             3.9.0                    py27_0  defaults        
                             3.9.0                    py34_0  defaults        
                             3.9.0                    py35_0  defaults        
                             3.10.0                   py27_0  defaults        
                             3.10.0                   py34_0  defaults        
                             3.10.0                   py35_0  defaults        


In [7]:
!conda create -n sj381-test simplejson=3.8.1 -y

Fetching package metadata .......
Solving package specifications: ..........

Package plan for installation in environment /Users/jbloom/anaconda/envs/sj381-test:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2j             |                0         3.0 MB
    setuptools-27.2.0          |           py27_0         522 KB
    simplejson-3.8.1           |           py27_0          80 KB
    pip-9.0.1                  |           py27_0         1.6 MB
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

    openssl:    1.0.2j-0     
    pip:        9.0.1-py27_0 
    python:     2.7.12-1     
    readline:   6.2-2        
    setuptools: 27.2.0-py27_0
    simplejson: 3.8.1-py27_0 
    sqlite:     3.13.0-0     
    tk:         8.5.18-0     
    wheel:    

In [8]:
!conda info -e

#
_test                    /Users/jbloom/anaconda/envs/_test
bootcamp                 /Users/jbloom/anaconda/envs/bootcamp
dd                       /Users/jbloom/anaconda/envs/dd
i13                      /Users/jbloom/anaconda/envs/i13
ipykernel_py2            /Users/jbloom/anaconda/envs/ipykernel_py2
numpy15                  /Users/jbloom/anaconda/envs/numpy15
py3k                     /Users/jbloom/anaconda/envs/py3k
seminar               *  /Users/jbloom/anaconda/envs/seminar
sj381-test               /Users/jbloom/anaconda/envs/sj381-test
root                     /Users/jbloom/anaconda



In [9]:
!ls /Users/jbloom/anaconda/envs/sj381-test/bin

2to3             easy_install-2.7 python-config    sqlite3
activate         idle             python2          tclsh8.5
c_rehash         openssl          python2-config   wheel
conda            pip              python2.7        wish8.5
deactivate       pydoc            python2.7-config
easy_install     python           smtpd.py


We could make this environment the default if we want to:
```bash
export PATH=~/anaconda/envs/sj381-test/bin:$PATH
```
And if we want to remove that environment:
```bash
conda remove -n sj381-test --all
```

In [12]:
!conda remove -n sj381-test --all -y


Package plan for package removal in environment /Users/jbloom/anaconda/envs/sj381-test:

The following packages will be REMOVED:

    openssl:    1.0.2j-0     
    pip:        9.0.1-py27_0 
    python:     2.7.12-1     
    readline:   6.2-2        
    setuptools: 27.2.0-py27_0
    simplejson: 3.8.1-py27_0 
    sqlite:     3.13.0-0     
    tk:         8.5.18-0     
    wheel:      0.29.0-py27_0
    zlib:       1.2.8-3      

Unlinking packages ...
[      COMPLETE      ]|###################################################| 100%


<center><h1> Command Line Parsing</h1></center>

<center>`python myawesomeprogram.py -o option1 -p parameter2 -Q -R`</center>
<p>
 **Goal**: build a command-line 'standalone' codebase in Python, w/ CL options & keywords
 
 **Solution**: `argparse`, which has been built in to Python 2.7 & above (if you don’t have it, you can get it with `pip argparse`)
 
* Allows for  user-friendly command line interfaces, and leaves it up to the code to determine what it was the user wanted.

* Also automatically generates help & usage messages and issues errors when invalid arguments are provided.

(Note on `optparse`: being replaced in favor of `argparse`)

In [13]:
import argparse

# Setting up a parser #


* First step for `argparse`: create parser object & tell it what arguments to expect. 
* It can then be used to process the command line arguments on runtime
* Parser class: `ArgumentParser`. Takes several arguments to set up the description used in the help text for the program & other global behaviors 
   
 <p>
See  http://www.doughellmann.com/PyMOTW/argparse/
</p>

In [15]:
%%writefile myfile.py
#!/usr/bin/env python
import argparse
parser = argparse.ArgumentParser(description='Sample Application')
print("hi")

Writing myfile.py


# Defining Arguments & Parsing

* Arguments can trigger different actions, specified by the action argument to `add_argument()`. 
* Several supported actions.
* Once all of the arguments are defined, you can parse the command line by passing a sequence of argument strings to `parse_args()`. 
* By default, arguments are taken from `sys.argv[1:]`, but you can also pass your own list.

In [18]:
%%file argparse_action.py
import argparse
parser = argparse.ArgumentParser(description='Sample Application')
parser.add_argument('required_arg_1', help='This positional argument is required')
parser.add_argument('required_arg_2', help='This positional argument is also required')
parser.add_argument('-s', action='store', dest='simple_value',
                    help='Store a simple value')
parser.add_argument('-c', action='store_const', dest='constant_value',
                    const='value-to-store',
                    help='Store a constant value')
parser.add_argument('-t', action='store_true', default=False,
                    dest='boolean_switch',
                    help='Set a switch to true')
parser.add_argument('-a', action='append', dest='collection',
                    default=[],
                    help='Add repeated values to a list',
                    )
parser.add_argument('-A', action='append_const', dest='const_collection',
                    const='value-1-to-append',
                    default=[],
                    help='Add different values to list')
parser.add_argument('-B', action='append_const', dest='const_collection',
                    const='value-2-to-append',
                    help='Add different values to list')
parser.add_argument('--version', action='version', version='%(prog)s 1.0')

results = parser.parse_args()
print('required_args    =', results.required_arg_1, results.required_arg_2)
print('simple_value     =', results.simple_value)
print('constant_value   =', results.constant_value)
print('boolean_switch   =', results.boolean_switch)
print('collection       =', results.collection)
print('const_collection =', results.const_collection)

Overwriting argparse_action.py


* `store`: Save the value, after optionally converting it to a different type (default)
* `store_const`: Save the value as defined as part of the argument specification, rather than a value that comes from the arguments being parsed
* `store_true`/`store_false`: Save the appropriate boolean value
* `append`: Save the value to a list.  Multiple values are saved if the argument is repeated
* `append_const`: Save a value defined in the argument specification to a list
* `version`: Prints version details about the program and then exits

There's also some other ways of doing arguement parsing that you might want to explore:

 * Click: http://click.pocoo.org/5/quickstart/#basic-concepts


In [21]:
!pip install click

In [29]:
%%writefile hello.py
import click
@click.command()
@click.option('--count', default=1, help='number of greetings')
@click.argument('name')
def hello(count, name):
    for x in range(count):
        click.echo('Hello %s!' % name)

if __name__ == '__main__':
    hello()

Overwriting hello.py


In [33]:
!python hello.py --help

Usage: hello.py [OPTIONS] NAME

Options:
  --count INTEGER  number of greetings
  --help           Show this message and exit.


In [35]:
!python hello.py --count=2 Josh

Hello Josh!
Hello Josh!


# Breakout! #

* Go to the breakout folder in: `../Breakouts/01_Versioning_Application_Building/`

* Work on the file `breakout1.py`.  Do not move or modify the other files, in the other folders, but you will need to use them.  (You may add files to these directories, if necessary)

* Build up a command line parser which allows the user to specify:
 - how many datapoints to generate
 - whether to plot with a filled in histogram or an outlined one
 - the title of the plot
 - And then have the plot be generated.

* We want to be able to run a command like:

```bash
python breakout1.py -t -n 200 -T "My Awesome Title"
```